In [ ]:
import qualified Data.Sequence as S
import Data.Foldable (toList)
import Control.Concurrent.Async (mapConcurrently)

input = makeB "01111010110010011"

makeB = S.fromList . fmap (\c -> case c of
    '1' -> True
    '0' -> False)

showB = toList . fmap (\b -> if b then '1' else '0')

dragon boolArr = boolArr S.>< (False S.<| (fmap not $ S.reverse boolArr))


showB (dragon (makeB "111100001010")) == "1111000010100101011110000"

In [ ]:
fillDisk input len
    | S.length input >= len = S.take len input
    | otherwise           = fillDisk (dragon input) len

filled = fillDisk input 272

checksum code acc
    | S.null   code              = checksum acc S.empty
    | S.length code `mod` 2 == 1 = code
    | otherwise                  = let
        (xy, rest) = S.splitAt 2 code
        x S.:< y'  = S.viewl xy
        y S.:< _   = S.viewl y'
        in checksum rest (acc S.|> (x==y))

showB $ checksum filled S.empty

In [ ]:
filled' = fillDisk input 35651584
S.length filled'

In [ ]:
{-
checksum8 code acc
    | S.null   code              = if (S.length acc `mod` 2 == 1)
        then acc
        else checksum8 acc S.empty
    | S.length code < 8          = let
        (xy, rest) = S.splitAt 2 code
        x S.:< y'  = S.viewl xy
        y S.:< _   = S.viewl y'
        in checksum8 rest (acc S.|> (x==y))
    | otherwise                  = let
        (as, rest) = S.splitAt 8 code
        a S.:< bs  = S.viewl as
        b S.:< cs  = S.viewl bs
        c S.:< ds  = S.viewl cs
        d S.:< es  = S.viewl ds
        e S.:< fs  = S.viewl es
        f S.:< gs  = S.viewl fs
        g S.:< hs  = S.viewl gs
        h S.:< _   = S.viewl hs
        value      = ((a==b)==(c==d))==((e==f)==(g==h))
        in checksum8 rest (acc S.|> value)
-}

take8 :: S.Seq a -> S.Seq (S.Seq a)
take8 sq = let
    (h, t) = S.splitAt 8 sq
    in if S.null t
        then h S.<| S.empty
        else h S.<| take8 t

checksum8 array | S.length array /= 8 = error "array not of length 8"
checksum8 array = let
    (as, rest) = S.splitAt 8 array
    a S.:< bs  = S.viewl as
    b S.:< cs  = S.viewl bs
    c S.:< ds  = S.viewl cs
    d S.:< es  = S.viewl ds
    e S.:< fs  = S.viewl es
    f S.:< gs  = S.viewl fs
    g S.:< hs  = S.viewl gs
    h S.:< _   = S.viewl hs
    value      = ((a==b)==(c==d))==((e==f)==(g==h))
    in value

concurrentCheckSum inputArr | S.length inputArr `mod` 2 == 1 = return inputArr
concurrentCheckSum inputArr = do
    let chunks = take8 inputArr
    result <- mapConcurrently (return . checksum8) chunks
    concurrentCheckSum result
    
concurrentCheckSum filled'


logBase 16 35651584

35651584 / 16^5

-- showB $ checksum8 filled' S.empty